<a href="https://colab.research.google.com/github/taehyeonk/pre-onboarding_project/blob/seondo/%EA%B8%B0%EC%97%85%EA%B3%BC%EC%A0%9C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

     |████████████████████████████████| 312 kB 8.0 MB/s 
     |████████████████████████████████| 212 kB 49.0 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 40.5 MB/s 
     |████████████████████████████████| 134 kB 61.7 MB/s 
     |████████████████████████████████| 127 kB 54.0 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 62.1 MB/s 
     |████████████████████████████████| 271 kB 63.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset

In [4]:
dataset = load_dataset("klue", "sts")

Downloading:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
dataset.keys()

dict_keys(['train', 'validation'])

In [16]:
train_data = dataset['train']
test_data = dataset['validation']

In [31]:
train_data['sentence2']

['숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 '시민들 스스로 자발적인 예방 노력을\xa0한 것은 아산 뿐만이 아니었다.',
 '사람들이 주로 네이버 메일을 쓰는 이유를 알려줘',
 '고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.',
 '호스트 응답이 늦었지만 개선될 것으로 보입니다.',
 '세계에서 우리만큼 오랜 역사와 문화를 공유하는 가까운 이웃이 없습니다.',
 '지하철을 탄다고 해도, 30분이면 그곳에 도착할 수 있어요!',
 '주말을 제외한 평일 오후 12시 30분부터 문예회관 공식 페이스북과 유튜브에서는 지역 예술인들이 중심이 된 서양음악, 국악, 댄스 등의 공연을 실시간으로 감상할 수 있다.',
 '조명등 낮에 켜놓으면 큰일나',
 '새로운 친구들을 만나고 싶을 때 추천합니다.',
 '식사 걸러봤자 살 빠지지도 않으니까 거르지 마세요.',
 '주방시설 상태도 좋고 욕실도 깨끗합니다.',
 '완벽에 가까울만큼 사진과 똑같은 장소였어요!',
 '다만, 관광지들과 거리가 조금 있습니다.',
 '약속 일정 변경은 당일이 아니라 미리미리 얘기하도록 하세요.',
 '신주쿠에서 두 정거장만 가면 히가시나카노 역에 도착할 수 있어요.',
 '시끌벅적한 런던 시내보다 훨씬 좋아요.',
 '시장에서 무슨 물건 사 오게?',
 '에어컨 쓰면 안 된다',
 '먼지 한 톨 없고 화장실도 광이 납니다.',
 '수압도 세고 따뜻한 물도 아주 잘 나옵니다.',
 '오늘 알람 몇 시로 예약해놨어?',
 '마지막으로, 국민의 안전한 삶과 튼튼한 국방, 평화를 향한 한결같은 의지를 담았습니다.',
 '화장실에 수건이랑 샴푸도 여러개 있었습니다.',
 '샤워는 수압이 너무 약해서 불편해요.',
 '그리고 무엇보다 방이 너무예쁘고 집전체가 너무너무 예뻐요.',
 '아울러 요양병원에서는 입원환자 중 원인미상의 폐렴환자를 격리해 진단검사를 실시한다.',
 '내일 기온은 영하야 영상이야?',
 '한편, 환경부와 한국환경공단